RESNET 18 IMPORT DIRECTLY

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# تنظیمات اولیه و انتقال به GPU اگر ممکن است
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# بارگذاری و پیش‌پردازش CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# بارگذاری مدل ResNet-18 و تغییر آخرین لایه به تعداد دسته‌های CIFAR-10
model = resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)  # چون CIFAR-10 ده دسته دارد
model = model.to(device)

# تعریف تابع از دست دادن و بهینه‌ساز
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# تابع محاسبه دقت (accuracy) در دیتاست تست
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# آموزش مدل
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Batch {i + 1}, Loss: {running_loss / 100:.3f}")
            running_loss = 0.0

    # ارزیابی مدل و محاسبه دقت در دیتاست تست
    model.eval()
    accuracy = calculate_accuracy(model, testloader)
    print(f"Epoch {epoch + 1} finished with accuracy: {accuracy:.2f}%")

print('آموزش به پایان رسید')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10
Batch 100, Loss: 2.070
Batch 200, Loss: 1.753
Batch 300, Loss: 1.606
Batch 400, Loss: 1.573
Batch 500, Loss: 1.517
Batch 600, Loss: 1.463
Batch 700, Loss: 1.424
Epoch 1 finished with accuracy: 50.26%
Epoch 2/10
Batch 100, Loss: 1.324
Batch 200, Loss: 1.313
Batch 300, Loss: 1.292
Batch 400, Loss: 1.269
Batch 500, Loss: 1.246
Batch 600, Loss: 1.248
Batch 700, Loss: 1.247
Epoch 2 finished with accuracy: 56.18%
Epoch 3/10
Batch 100, Loss: 1.128
Batch 200, Loss: 1.123
Batch 300, Loss: 1.107
Batch 400, Loss: 1.110
Batch 500, Loss: 1.077
Batch 600, Loss: 1.090
Batch 700, Loss: 1.079
Epoch 3 finished with accuracy: 60.45%
Epoch 4/10
Batch 100, Loss: 0.992
Batch 200, Loss: 1.011
Batch 300, Loss: 0.969
Batch 400, Loss: 0.983
Batch 500, Loss: 0.971
Batch 600, Loss: 0.982
Batch 700, Loss: 0.956
Epoch 4 finished with accuracy: 62.78%
Epoch 5/10
Batch 100, Loss: 0.882
Batch 200, Loss: 0.878
Batch 300, Loss: 0.889


KeyboardInterrupt: 

RESNET 18 FROM SCRATCH

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# تعریف بلوک پایه‌ای ResNet
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# تعریف مدل ResNet
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.functional.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# ساخت مدل ResNet-18
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# تنظیمات اولیه و انتقال به GPU اگر ممکن است
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18().to(device)

# بارگذاری و پیش‌پردازش CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# تعریف تابع از دست دادن و بهینه‌ساز
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# تابع محاسبه دقت (accuracy) در دیتاست تست
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# آموزش مدل
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Batch {i + 1}, Loss: {running_loss / 100:.3f}")
            running_loss = 0.0

    # ارزیابی مدل و محاسبه دقت در دیتاست تست
    model.eval()
    accuracy = calculate_accuracy(model, testloader)
    print(f"Epoch {epoch + 1} finished with accuracy: {accuracy:.2f}%")

print('آموزش به پایان رسید')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10
Batch 100, Loss: 1.959
Batch 200, Loss: 1.557
Batch 300, Loss: 1.415
Batch 400, Loss: 1.354
Batch 500, Loss: 1.280
Batch 600, Loss: 1.196
Batch 700, Loss: 1.125
Epoch 1 finished with accuracy: 61.37%
Epoch 2/10
Batch 100, Loss: 1.025
Batch 200, Loss: 0.991
Batch 300, Loss: 0.937
Batch 400, Loss: 0.926
Batch 500, Loss: 0.901
Batch 600, Loss: 0.876
Batch 700, Loss: 0.859
Epoch 2 finished with accuracy: 71.05%
Epoch 3/10
Batch 100, Loss: 0.728
Batch 200, Loss: 0.708
Batch 300, Loss: 0.713
Batch 400, Loss: 0.699
Batch 500, Loss: 0.682
Batch 600, Loss: 0.684
Batch 700, Loss: 0.695
Epoch 3 finished with accuracy: 73.50%
Epoch 4/10
Batch 100, Loss: 0.593
Batch 200, Loss: 0.580
Batch 300, Loss: 0.551
Batch 400, Loss: 0.576
Batch 500, Loss: 0.569
Batch 600, Loss: 0.569
Batch 700, Loss: 0.552
Epoch 4 finished with accuracy: 77.35%
Epoch 5/10
Batch 100, Loss: 0.445
Batch 200, Loss: 0.465
Batch 300, Loss: 0.450


KeyboardInterrupt: 

RESNET 18 WITH AUTO FORWARD AND BACKWARD PROPAGATION

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

filename = "multi_part_resnet_18.txt"
with open(filename, "w") as file:
  file.write("accuracies values")
# تنظیمات اولیه و انتقال به GPU اگر ممکن است
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تعریف بلوک پایه‌ای ResNet
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# پارتیشن‌های مدل
class Partition1(nn.Module):
    def __init__(self):
        super(Partition1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        return out

class Partition2(nn.Module):
    def __init__(self, in_channels):
        super(Partition2, self).__init__()
        self.layer1 = self._make_layer(BasicBlock, in_channels, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 64, 128, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        return out

class Partition3(nn.Module):
    def __init__(self, in_channels):
        super(Partition3, self).__init__()
        self.layer3 = self._make_layer(BasicBlock, in_channels, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 256, 512, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer3(x)
        out = self.layer4(out)
        return out

class Partition4(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(Partition4, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        out = self.avgpool(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# ایجاد پارتیشن‌ها
partition1 = Partition1().to(device)
partition2 = Partition2(64).to(device)
partition3 = Partition3(128).to(device)
partition4 = Partition4(512).to(device)

# تابع انتقال بین پارتیشن‌ها
def forward_distributed(x):
    x = partition1(x)
    x = partition2(x)
    x = partition3(x)
    x = partition4(x)
    return x

# بارگذاری و پیش‌پردازش CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# تعریف تابع از دست دادن و بهینه‌ساز
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(partition1.parameters()) + list(partition2.parameters()) +
                      list(partition3.parameters()) + list(partition4.parameters()), lr=0.001, momentum=0.9)

# تابع محاسبه دقت (accuracy) در دیتاست تست
def calculate_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = forward_distributed(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# آموزش مدل
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = forward_distributed(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Batch {i + 1}, Loss: {running_loss / 100:.3f}")
            running_loss = 0.0

    # ارزیابی مدل و محاسبه دقت در دیتاست تست
    accuracy = calculate_accuracy()
    print(f"Epoch {epoch + 1} finished with accuracy: {accuracy:.2f}%")
    file.write(f"{epoch +1} - ${accuracy:.2f}")
print('آموزش به پایان رسید')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10
Batch 100, Loss: 1.933
Batch 200, Loss: 1.562
Batch 300, Loss: 1.406
Batch 400, Loss: 1.319
Batch 500, Loss: 1.248
Batch 600, Loss: 1.175
Batch 700, Loss: 1.122
Epoch 1 finished with accuracy: 63.01%
Epoch 2/10
Batch 100, Loss: 0.983
Batch 200, Loss: 0.943
Batch 300, Loss: 0.917
Batch 400, Loss: 0.895
Batch 500, Loss: 0.897
Batch 600, Loss: 0.824
Batch 700, Loss: 0.836
Epoch 2 finished with accuracy: 70.54%
Epoch 3/10
Batch 100, Loss: 0.743
Batch 200, Loss: 0.708
Batch 300, Loss: 0.682
Batch 400, Loss: 0.676
Batch 500, Loss: 0.681
Batch 600, Loss: 0.679
Batch 700, Loss: 0.656
Epoch 3 finished with accuracy: 75.63%
Epoch 4/10
Batch 100, Loss: 0.557
Batch 200, Loss: 0.534
Batch 300, Loss: 0.546
Batch 400, Loss: 0.570
Batch 500, Loss: 0.550
Batch 600, Loss: 0.581
Batch 700, Loss: 0.539
Epoch 4 finished with accuracy: 77.53%
Epoch 5/10
Batch 100, Loss: 0.458
Batch 200, Loss: 0.477
Batch 300, Loss: 0.455


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# تنظیمات اولیه و انتقال به GPU اگر ممکن است
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تعریف بلوک پایه‌ای ResNet
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# پارتیشن‌های مدل
class Partition1(nn.Module):
    def __init__(self):
        super(Partition1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        return out

class Partition2(nn.Module):
    def __init__(self, in_channels):
        super(Partition2, self).__init__()
        self.layer1 = self._make_layer(BasicBlock, in_channels, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 64, 128, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        return out

class Partition3(nn.Module):
    def __init__(self, in_channels):
        super(Partition3, self).__init__()
        self.layer3 = self._make_layer(BasicBlock, in_channels, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 256, 512, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer3(x)
        out = self.layer4(out)
        return out

class Partition4(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(Partition4, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        out = self.avgpool(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# ایجاد پارتیشن‌ها
partition1 = Partition1().to(device)
partition2 = Partition2(64).to(device)
partition3 = Partition3(128).to(device)
partition4 = Partition4(512).to(device)

# بارگذاری و پیش‌پردازش CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# تعریف تابع از دست دادن و بهینه‌ساز
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(partition1.parameters()) + list(partition2.parameters()) +
                      list(partition3.parameters()) + list(partition4.parameters()), lr=0.001, momentum=0.9)

# شبیه‌سازی فرآیند forward و backward با متغیرهای لوکال
for epoch in range(1):  # یک epoch برای تست
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # Forward pass با متغیرهای لوکال
        output_layer1 = partition1(inputs)
        output_layer2 = partition2(output_layer1.detach())
        output_layer3 = partition3(output_layer2.detach())
        output_layer4 = partition4(output_layer3.detach())

        # محاسبه Loss
        loss = criterion(output_layer4, labels)

        # Backward pass به صورت دستی
        optimizer.zero_grad()

        # backward برای مرحله آخر
        output_layer4.backward(torch.ones_like(output_layer4), retain_graph=True)

        # backward برای مراحل میانی به صورت دستی
        output_layer3.backward(torch.ones_like(output_layer3), retain_graph=True)
        output_layer2.backward(torch.ones_like(output_layer2), retain_graph=True)
        output_layer1.backward(torch.ones_like(output_layer1))

        # به‌روزرسانی وزن‌ها
        optimizer.step()

        if i % 100 == 0:
            print(f"Batch {i}, Loss: {loss.item():.4f}")


Files already downloaded and verified
Batch 0, Loss: 2.3641
Batch 100, Loss: 2.2966
Batch 200, Loss: 2.3044
Batch 300, Loss: 2.3019
Batch 400, Loss: 2.3020
Batch 500, Loss: 2.3012
Batch 600, Loss: 2.2998
Batch 700, Loss: 2.3019


## code with custom forward and backward

RESNET-18 WITH CUSTOM FORWARD AND BACKWARD PROPATION

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# تنظیمات اولیه و انتقال به GPU اگر ممکن است
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
list_accuracies = []


# تعریف بلوک پایه‌ای ResNet
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# پارتیشن‌های مدل
class Partition1(nn.Module):
    def __init__(self):
        super(Partition1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        return out

class Partition2(nn.Module):
    def __init__(self, in_channels):
        super(Partition2, self).__init__()
        self.layer1 = self._make_layer(BasicBlock, in_channels, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 64, 128, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        return out

class Partition3(nn.Module):
    def __init__(self, in_channels):
        super(Partition3, self).__init__()
        self.layer3 = self._make_layer(BasicBlock, in_channels, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 256, 512, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer3(x)
        out = self.layer4(out)
        return out

class Partition4(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(Partition4, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        out = self.avgpool(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# ایجاد پارتیشن‌ها
partition1 = Partition1().to(device)
partition2 = Partition2(64).to(device)
partition3 = Partition3(128).to(device)
partition4 = Partition4(512).to(device)

# بارگذاری و پیش‌پردازش CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# تعریف تابع از دست دادن و بهینه‌ساز
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(list(partition1.parameters()) + list(partition2.parameters()) +
                      list(partition3.parameters()) + list(partition4.parameters()), lr=0.01, momentum=0.9)

# تابع محاسبه دقت (accuracy) در دیتاست تست
def calculate_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            output_layer1 = partition1(images)
            output_layer2 = partition2(output_layer1)
            output_layer3 = partition3(output_layer2)
            output_layer4 = partition4(output_layer3)
            _, predicted = torch.max(output_layer4, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# آموزش مدل
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # Forward pass بدون .detach()
        output_layer1 = partition1(inputs)
        output_layer2 = partition2(output_layer1)
        output_layer3 = partition3(output_layer2)
        output_layer4 = partition4(output_layer3)

        # محاسبه Loss
        loss = criterion(output_layer4, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Batch {i + 1}, Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    # ارزیابی مدل و محاسبه دقت در دیتاست تست
    accuracy = calculate_accuracy()
    print(f"Epoch {epoch + 1} finished with accuracy: {accuracy:.2f}%")
    list_accuracies.append(accuracy)
print('آموزش به پایان رسید')



filename = "multi_part_resnet18.txt"
# Open the file in write mode
with open(filename, "w") as file:
  file.write(f'accuracies: {list_accuracies}')



Files already downloaded and verified
Files already downloaded and verified
Epoch 1/5
Batch 100, Loss: 1.8725
Batch 200, Loss: 1.5423
Batch 300, Loss: 1.3653
Batch 400, Loss: 1.2401
Batch 500, Loss: 1.1528
Batch 600, Loss: 1.0644
Batch 700, Loss: 0.9978
Epoch 1 finished with accuracy: 65.56%
Epoch 2/5
Batch 100, Loss: 0.8432
Batch 200, Loss: 0.7799
Batch 300, Loss: 0.7822
Batch 400, Loss: 0.7575
Batch 500, Loss: 0.7300
Batch 600, Loss: 0.7097
Batch 700, Loss: 0.6991
Epoch 2 finished with accuracy: 76.17%
Epoch 3/5
Batch 100, Loss: 0.5960
Batch 200, Loss: 0.6353
Batch 300, Loss: 0.5930
Batch 400, Loss: 0.5867
Batch 500, Loss: 0.5796
Batch 600, Loss: 0.5495
Batch 700, Loss: 0.5443
Epoch 3 finished with accuracy: 80.95%
Epoch 4/5
Batch 100, Loss: 0.4675
Batch 200, Loss: 0.4665
Batch 300, Loss: 0.4705
Batch 400, Loss: 0.4517
Batch 500, Loss: 0.4797
Batch 600, Loss: 0.4663
Batch 700, Loss: 0.4279
Epoch 4 finished with accuracy: 80.38%
Epoch 5/5
Batch 100, Loss: 0.4042
Batch 200, Loss: 0.376